In [1]:
!pip install -r requirements.txt

In [2]:
# Bibliothèques standard
import os, requests, zipfile

# Data science et manipulation de données
import pandas as pd, numpy as np, geopandas as gpd

# Accès S3
import s3fs


# Fonctions et styles personnalisés importés de visualisation.py
from visualisation import (
    charte_graphique,
    charte_graphique2,
    charte_graphique3,
    dictionnaire_geo,
    tracer_evolution_taux,
    tracer_evolution_taux_relatif_lisse,
    boxplot_indicateur_par_saison,
    evolution_indicateur_animation,
    animer_evolution_densite,
    evolution_idf_animation,
    get_increase,
    tri_occurrence,
    plot_histogram,
    camembert,
    nb_lignes_traitant,
    filter_rows_with_keyword,
    count_crime_keywords,
    évolution_indicateur,
    évolution_indicateur,
)

In [3]:
fs = s3fs.S3FileSystem(client_kwargs={"endpoint_url": "https://minio.lab.sspcloud.fr"})
MY_BUCKET = "anhlinh"

FILE_PATH_S3_DEP = f"{MY_BUCKET}/diffusion/df_indicateurs_dep.csv"
with fs.open(FILE_PATH_S3_DEP, "rb") as file_in_dep:
    df_indicateurs_dep = pd.read_csv(file_in_dep)

# Récupération de la base de données législative
FILE_PATH_S3_LODA = f"{MY_BUCKET}/diffusion/df_loda.csv"
with fs.open(FILE_PATH_S3_LODA, "rb") as file_in_loda:
    df_loda = pd.read_csv(file_in_loda)


,Unnamed: 0,Titre,ID,Date,Nature,Etat,Origine,Date Publication,Année,Mois
0,0,Arrêté du 31 décembre 2008 relatif aux modalit...,LEGITEXT000020083722_27-05-2024,2008-12-31,ARRETE,VIGUEUR,LEGI,2009-01-01T00:00:00.000+0000,2009,1
1,1,Arrêté du 31 décembre 2008 portant création d'...,LEGITEXT000020167092_30-01-2009,2008-12-31,ARRETE,VIGUEUR,LEGI,2009-01-29T00:00:00.000+0000,2009,1
2,2,Décret n° 2008-1549 du 31 décembre 2008 portan...,LEGITEXT000020080924_02-01-2009,2008-12-31,DECRET,VIGUEUR,LEGI,2009-01-01T00:00:00.000+0000,2009,1
3,3,Arrêté du 31 décembre 2008 relatif aux modalit...,LEGITEXT000020083722_18-12-2015,2008-12-31,ARRETE,VIGUEUR,LEGI,2009-01-01T00:00:00.000+0000,2009,1
4,4,Arrêté du 31 décembre 2008 relatif aux modalit...,LEGITEXT000049943833_01-10-2024,2008-12-31,ARRETE,VIGUEUR,LEGI,2009-01-01T00:00:00.000+0000,2009,1


In [10]:
df_loda_reg = df_loda.drop([ "Unnamed: 0", "ID", "Date", "Nature", "Etat", "Origine", "Date Publication", "Mois"], axis = 1)
df_loda_reg.head()

,Titre,Année
0,Arrêté du 31 décembre 2008 relatif aux modalit...,2009
1,Arrêté du 31 décembre 2008 portant création d'...,2009
2,Décret n° 2008-1549 du 31 décembre 2008 portan...,2009
3,Arrêté du 31 décembre 2008 relatif aux modalit...,2009
4,Arrêté du 31 décembre 2008 relatif aux modalit...,2009


In [4]:
df_indicateurs_dep.head()
df_indicateurs_reg = df_indicateurs_dep.drop([ "Unnamed: 0", ], axis = 1)

,Unnamed: 0,Année,Département,Indicateur,Nombre,Population,Taux (/10 000),Nom Département,Superficie (km2),Densité
0,0,1996,01,Homicides,17.0,499310,0.340470,Ain,5762,86.655675
1,1,1996,02,Homicides,14.0,538017,0.260215,Aisne,7362,73.080277
2,2,1996,03,Homicides,11.0,349376,0.314847,Allier,7340,47.598910
3,3,1996,04,Homicides,6.0,137453,0.436513,Alpes-de-Haute-Provence,6925,19.848809
4,4,1996,05,Homicides,5.0,118526,0.421848,Hautes-Alpes,5549,21.359885
